In [79]:
# import numpy as np
# from IPython.display import Image
# Image(filename='before_photo.jpg') 

import numpy as np
from PIL import Image
from scipy.ndimage import minimum_filter

In [72]:
# # [old
# before = Image.open("before_photo.jpg") 

# width, height = before.size

# rgb = []
# for w in range(width):
#     for h in range(height):
#         r,g,b = before.getpixel((w,h))

#         rgb.append([r,g,b])


# min_rgb = []
# options = [0,1,2]
# zeros = [0,0,0]
# for i in range(len(rgb)):
#     min_index = np.argmin(rgb[i])
#     zeros[min_index] = min(rgb[i])
#     min_rgb.append(zeros)
#     zeros = [0,0,0]

# dark_channel = np.empty((height, width), dtype = object)
# for i in range(height):
#     for j in range(width):
#         dark_channel[i][j] = min_rgb[j + width * i]


# dark_channel = Image.fromarray(dark_channel, "RGB")
# dark_channel.save("dark channel.png")
# dark_channel.show()


In [90]:
before = Image.open("before_photo.jpg") 

width, height = before.size

rgb = []
for w in range(width):
    for h in range(height):
        r,g,b = before.getpixel((w,h))

        rgb.append([r,g,b])


min_rgb = []
for i in range(len(rgb)):
    min_rgb.append(min(rgb[i]))


dark_channel = np.empty((height, width))
for i in range(height):
    for j in range(width):
        dark_channel[i][j] = min_rgb[j + width * i]


# dark_channel = Image.fromarray(dark_channel, "L")
# dark_channel.save("dark channel.png")
# dark_channel.show()

kernel_size = 15  # You can adjust the kernel size as needed
dark_channel_filtered = minimum_filter(dark_channel, size=kernel_size)
dark_channel_image = Image.fromarray(dark_channel_filtered, "L")
dark_channel_image.save("dark channel.png")
dark_channel_image.show()

In [76]:
print(width, height)

550 413


In [135]:
image = Image.open('before_photo.jpg')  # Replace with your image path

# Convert the image to a NumPy array
image_np = np.array(image)

# Get the minimum intensity across the R, G, B channels for each pixel
dark_channel = np.min(image_np, axis=2)  # This creates a grayscale image

# # Apply a minimum filter using scipy.ndimage.minimum_filter
kernel_size = 15  # You can adjust the kernel size as needed
dark_channel_filtered = minimum_filter(dark_channel, size=kernel_size)

# # Convert the result back to a PIL image
dark_channel_image = Image.fromarray(dark_channel_filtered)

# # Save or show the result
dark_channel_image.save('dark channel.png')
dark_channel_image.show()

(413, 550)


In [152]:
# equation
transmissionMap = 1 - 0.95 * dark_channel_filtered

darkChannelBrightestPixel = 0
currentBrightest = 0
globalAtmoshpericLight = []
c = 0
for i in range(width):
    for j in range(height):
        if dark_channel_filtered[j][i] > currentBrightest:
            currentBrightest = dark_channel_filtered[j][i]
            darkChannelBrightestPixel = c
        c += 1

column = c % width - 1
row = c // width - 1

globalAtmoshpericLight = image_np[row][column]

sceneRadiance = ((image_np - globalAtmoshpericLight)/max(transmissionMap.any(), 0.1) + globalAtmoshpericLight).astype(int)
after = Image.fromarray(sceneRadiance, "RGB")
after.save("after.png")
after.show()

In [156]:
import numpy as np
from PIL import Image

# Calculate transmission map and ensure it's clipped to a minimum value
transmissionMap = np.clip(1 - 0.95 * dark_channel_filtered, 0.1, 1)

# Find the pixel with the brightest value in the dark channel
darkChannelBrightestPixel = 0
currentBrightest = 0
for i in range(width):
    for j in range(height):
        if dark_channel_filtered[j][i] > currentBrightest:
            currentBrightest = dark_channel_filtered[j][i]
            darkChannelBrightestPixel = (j, i)

# Extract the atmospheric light from the original image using the brightest pixel from the dark channel
row, column = darkChannelBrightestPixel
globalAtmosphericLight = image_np[row, column]  # This will be a 3-channel RGB value

# Debug: Print the global atmospheric light to check the value
print("Global Atmospheric Light (RGB):", globalAtmosphericLight)

# Ensure atmospheric light is reshaped and broadcasted correctly
atmospheric_light_broadcast = np.reshape(globalAtmosphericLight, (1, 1, 3))

# Debug: Print a small part of the transmission map
print("Transmission Map Sample (Top-Left):", transmissionMap[0:5, 0:5])

# Recover the scene radiance, applying the transmission map per channel (RGB)
sceneRadiance = (image_np - atmospheric_light_broadcast) / np.maximum(transmissionMap[:, :, np.newaxis], 0.1) + atmospheric_light_broadcast

# Debug: Print the scene radiance values before clipping
print("Scene Radiance Sample (Top-Left):", sceneRadiance[0:5, 0:5])

# Clip values to ensure they're within valid image range (0-255)
sceneRadiance = np.clip(sceneRadiance, 0, 255).astype(np.uint8)

# Save and display the result
after = Image.fromarray(sceneRadiance, "RGB")
after.save("after.png")
after.show()


Global Atmospheric Light (RGB): [231 229 230]
Transmission Map Sample (Top-Left): [[0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1]
 [0.1 0.1 0.1 0.1 0.1]]
Scene Radiance Sample (Top-Left): [[[2361. 2439. 2550.]
  [2361. 2439. 2550.]
  [2371. 2449. 2560.]
  [2371. 2449. 2560.]
  [2371. 2449. 2560.]]

 [[2361. 2439. 2550.]
  [2361. 2439. 2550.]
  [2371. 2449. 2560.]
  [2371. 2449. 2560.]
  [2371. 2449. 2560.]]

 [[2361. 2449. 2540.]
  [2361. 2449. 2540.]
  [2371. 2459. 2550.]
  [2371. 2459. 2550.]
  [2371. 2459. 2550.]]

 [[2361. 2449. 2540.]
  [2361. 2449. 2540.]
  [2371. 2459. 2550.]
  [2371. 2459. 2550.]
  [2371. 2459. 2550.]]

 [[2361. 2449. 2540.]
  [2361. 2449. 2540.]
  [2371. 2459. 2550.]
  [2371. 2459. 2550.]
  [2371. 2459. 2550.]]]


In [7]:
attenuatuionCoefficient = -0.007
sceneDepth = 300
transitionMap = np.e ** -(attenuatuionCoefficient * sceneDepth)



print(r,g,b)

188 194 206
